# Volume 3: Discrete Hidden Markov Models
    Matthew Mella
    02/27/2024

In [57]:
import numpy as np
import copy
import string
import codecs
from hmmlearn import hmm

# Problems 1-5
This is the HMM class that you will be adding functions to throughout the lab.

In [43]:
class HMM(object):
    """
    Finite state space hidden Markov model.
    """
    
    # Problem 1
    def __init__(self, A, B, pi):
        """
        Initialize an HMM with parameters A, B, and pi.
        """
        self.A = A
        self.B = B
        self.pi = pi
    
    
    # Problem 2
    def forward_pass(self, z):
        """
        Compute the forward probability matrix.

        Parameters
        ----------
        z : ndarray of shape (T,)
            The observation sequence

        Returns
        -------
        alpha : ndarray of shape (T, n)
            The forward probability matrix
        """

        T = len(z)
        n = self.A.shape[0]

        # Initialize alpha
        alpha = np.zeros((T, n))

        # Compute alpha_0
        alpha[0] = self.pi * self.B[z[0], :]

        # Compute alpha_t
        for t in range(1, T):
            alpha[t] = self.B[z[t], :] * (alpha[t-1] @ self.A.T)

        return alpha
    
    
    # Problem 4
    def backward_pass(self, z):
        """
        Compute the backward probability matrix and gamma values.

        Parameters
        ----------
        z : ndarray of shape (T,)
            The observation sequence

        Returns
        -------
        beta : ndarray of shape (T, n)
            The backward probability matrix
        gamma : ndarray of shape (T, n)
            The state probability matrix
        """

        # Compute alpha with forward_pass
        alpha = self.forward_pass(z)

        T = len(z)
        n = self.A.shape[0]

        # Initialize beta
        beta = np.zeros((T,n))

        # Set beta_T
        beta[-1, :] = 1

        # Compute beta_t
        for t in range(T - 2, -1, -1):
            beta[t, :] = self.A.T @ (self.B[z[t + 1], :] * beta[t + 1, :])

        # Compute gamma
        gamma = alpha * beta / np.sum(alpha[-1, :])

        # Return beta and gamma
        return beta, gamma
    
    # Problem 5
    def viterbi_algorithm(self, z):
        """
        Compute the most likely hidden state sequence using the Viterbi algorithm.

        Parameters
        ----------
        z : ndarray of shape (T,)
            The observation sequence

        Returns
        -------
        x*: ndarray of shape (T,)
            The most likely state sequence
        """
        T = len(z)
        n = self.A.shape[0]

        # Initialize eta
        eta = np.zeros((T, n))

        # Compute eta_0
        eta[0] = self.B[z[0], :] * self.pi

        # Compute eta_t
        for t in range(1, T):
            eta[t] = np.max(self.B[z[t], :, None] * self.A * eta[t-1], axis = 1)


        # Backtrack to find the most likely state sequence
        xstar = np.zeros(T, dtype=int)
        
        # Set the last state
        xstar[-1] = np.argmax(eta[-1])

        # Compute xstar_t
        for t in range(T - 1, 0, -1):
            xstar[t-1] = np.argmax(self.A[xstar[t],:] * eta[t-1])

        # Return the most likely state sequence
        return xstar

# Problem 2 test case

Use the following HMM and code to test your HMM class.
Compare the output to `forward_pass` with the lab pdf.

In [46]:
pi = np.array([.6, .4])
A = np.array([[.7, .4],[.3, .6]])
B = np.array([[.1,.7],[.4, .2],[.5, .1]])
z_example = np.array([0, 1, 0, 2])
example_hmm = HMM(A, B, pi)

alpha=example_hmm.forward_pass(z_example)
print(np.sum(alpha[-1,:]))

0.009629599999999999


# Problem 3

Consider the following (very simplified) model of the price of a stock over time as an HMM.
The observation states will be the change in the value of the stock.
For simplicity, we will group these into five values: large decrease, small decrease, no change, small increase, large increase, labeled as integers from 0 to 4.
The hidden state will be the overall trends of the market.
We'll consider the market to have three possible states: declining in value (bear market), not changing in value (stagnant), and increasing in value (bull market), labeled as integers from 0 to 2.
Let the HMM modeling this scenario have parameters
$$
\boldsymbol\pi=\begin{bmatrix}
1/3 \\ 1/3 \\ 1/3
\end{bmatrix},
\quad
A=\begin{bmatrix}
0.5 & 0.3 & 0 \\
0.5 & 0.3 & 0.3 \\
0 & 0.4 & 0.7
\end{bmatrix},
\quad
B=\begin{bmatrix}
0.3 & 0.1 & 0 \\
0.3 & 0.2 & 0.1 \\
0.3 & 0.4 & 0.3 \\
0.1 & 0.2 & 0.4 \\
0 & 0.1 & 0.2
\end{bmatrix}
$$
The file `stocks.npy` contains a sequence of 50 observations drawn from this HMM.
What is the probability of this observation sequence given these model parameters?
Use your implementation of the forward pass algorithm from Problem 2 to find the answer.
Note that the answer is very small, because there are lots of possible observation sequences.

In [47]:
# HMM parameter setup
pi = np.array([1/3, 1/3, 1/3])
A = np.array([
    [0.5, 0.3, 0.0],
    [0.5, 0.3, 0.3],
    [0.0, 0.4, 0.7]
])
B = np.array([
    [0.3, 0.1, 0.0],
    [0.3, 0.2, 0.1],
    [0.3, 0.4, 0.3],
    [0.1, 0.2, 0.4],
    [0.0, 0.1, 0.2]
])

hmm = HMM(A, B, pi)

data = np.load('stocks.npy')

alpha = hmm.forward_pass(data)

print(np.sum(alpha[-1,:]))

6.67111511453778e-34


# Problem 4

Create a method `backward_pass` in your HMM class to implement the backward pass algorithm.
This function should accept the observation sequence $\mathbf{z}$ and return two arrays of the $\beta_t(i)$ and $\gamma_t(i)$ values.

Test your function on the example HMM, and compare the output with the lab pdf.

With your function and the stock model from Problem 3, answer the following question: given the observation sequence in `stocks.npy`, what is the most likely initial hidden state $X_0$?

In [7]:
# Test case; compare your output with what is in the lab pdf
beta, gamma = example_hmm.backward_pass(z_example)
print(beta)
print(gamma)

[[0.0302  0.02792]
 [0.0812  0.1244 ]
 [0.38    0.26   ]
 [1.      1.     ]]
[[0.18816981 0.81183019]
 [0.51943175 0.48056825]
 [0.22887763 0.77112237]
 [0.8039794  0.1960206 ]]


In [50]:
# HMM parameter setup
pi = np.array([1/3, 1/3, 1/3])
A = np.array([
    [0.5, 0.3, 0.0],
    [0.5, 0.3, 0.3],
    [0.0, 0.4, 0.7]
])
B = np.array([
    [0.3, 0.1, 0.0],
    [0.3, 0.2, 0.1],
    [0.3, 0.4, 0.3],
    [0.1, 0.2, 0.4],
    [0.0, 0.1, 0.2]
])

hmm = HMM(A, B, pi)

data = np.load('stocks.npy')

beta, gamma = hmm.backward_pass(data)

# find most likely initial hidden state
initial_state = np.argmax(gamma[0])
print("The most likely initial hidden state is", initial_state)


The most likely initial hidden state is 1


# Problem 5
Creating a method `viterbi_algorithm` in your HMM class to implement the Viterbi algorithm.
This function should accept the observation sequence $\mathbf{z}$ and return the most likely state sequence $\mathbf{x}^*$.

Test your function on the example HMM and compare output with the lab pdf.

Apply your function to the stock market HMM from Problem 3.
With the observaition sequence from `stocks.npy`, what is the most likely sequence of hidden states?
Is the initial state of the most likely sequence the same as the most likely initial state you found in Problem 4?

In [70]:
# Test case
xstar = example_hmm.viterbi_algorithm(z_example)
print(xstar)

[1 1 1 0]


In [69]:
# HMM parameter setup
pi = np.array([1/3, 1/3, 1/3])
A = np.array([
    [0.5, 0.3, 0.0],
    [0.5, 0.3, 0.3],
    [0.0, 0.4, 0.7]
])
B = np.array([
    [0.3, 0.1, 0.0],
    [0.3, 0.2, 0.1],
    [0.3, 0.4, 0.3],
    [0.1, 0.2, 0.4],
    [0.0, 0.1, 0.2]
])

hmm = HMM(A, B, pi)

data = np.load('stocks.npy')

beta, gamma = hmm.backward_pass(data)

# find most likely sequence
sequence = hmm.viterbi_algorithm(data)

print("The most likely sequence is:")
print(sequence)


The most likely sequence is:
[0 0 0 0 0 1 2 2 2 2 2 2 2 2 2 2 1 2 2 2 2 2 2 2 2 2 2 1 0 0 0 1 2 2 2 2 2
 2 2 2 2 2 2 1 0 0 0 0 0 1]


For problem 4, we find the most likely state is 1 for t0, but in this case, as we consider the sequence as a whole, the most likely state starts with 0

# Problem 6

Train a `hmmlearn.hmm.CategoricalHMM` on `declaration.txt`. Use `N=2` states and `M=len(set(obs))=27` observation values (26 lower case characters and 1 whitespace character).
Use `n_iter=200` and `tol=1e-4`.

Once the learning algorithm converges, analyze the state observation matrix $B$. Note which rows correspond to the largest and smallest probability values in each column of $B$, and check the corresponding characters. The HMM should have detected a vowel state and a consonant state.

In [58]:
def vec_translate(a, my_dict):
    # translate numpy array from symbols to state numbers or vice versa
    return np.vectorize(my_dict.__getitem__)(a)

def prep_data(filename):
    """
    Reads in the file and prepares it for use in an HMM.
    Returns:
        symbols (dict): a dictionary that maps characters to their integer values
        obs_sequence (ndarray): an array of integers representing the read-in text
    """
    # Get the data as a single string
    with codecs.open(filename, encoding='utf-8') as f:
        data=f.read().lower()  # and convert to all lower case
    # remove punctuation and newlines
    remove_punct_map = {ord(char): 
                        None for char in string.punctuation+"\n\r"}
    data = data.translate(remove_punct_map)
    # make a list of the symbols in the data
    symbols = sorted(list(set(data)))
    # convert the data to a NumPy array of symbols
    a = np.array(list(data))
    # make a conversion dictionary from symbols to state numbers
    symbols_to_obsstates = {x:i for i,x in enumerate(symbols)}
    # convert the symbols in a to state numbers
    obs_sequence = vec_translate(a,symbols_to_obsstates)
    return symbols, obs_sequence

In [64]:
symbols, obs = prep_data('declaration.txt')
h = hmm.CategoricalHMM(n_components=2, n_iter=200, tol=1e-4)
h.fit(obs.reshape(-1,1))

CategoricalHMM(n_components=2, n_features=27, n_iter=200,
               random_state=RandomState(MT19937) at 0x105A0E840, tol=0.0001)

In [66]:
B = h.emissionprob_.T
for i in range(len(B)):
    print(u"{}, {:0.4f}, {:0.4f}"
                .format(symbols[i], *B[i,:]))

 , 0.2987, 0.0503
a, 0.1319, 0.0000
b, 0.0000, 0.0226
c, 0.0000, 0.0437
d, 0.0000, 0.0599
e, 0.2376, 0.0000
f, 0.0000, 0.0428
g, 0.0000, 0.0309
h, 0.0000, 0.0829
i, 0.1242, 0.0000
j, 0.0000, 0.0038
k, 0.0004, 0.0030
l, 0.0000, 0.0542
m, 0.0000, 0.0342
n, 0.0000, 0.1147
o, 0.1376, 0.0036
p, 0.0000, 0.0328
q, 0.0000, 0.0014
r, 0.0000, 0.1009
s, 0.0000, 0.1135
t, 0.0000, 0.1520
u, 0.0578, 0.0000
v, 0.0000, 0.0176
w, 0.0000, 0.0230
x, 0.0000, 0.0021
y, 0.0117, 0.0092
z, 0.0000, 0.0010


I notice that, in column 0, the vowels all have a relatively large value, but their value is 0 for column 1. Similarly, all of the constants have a zero value in column 0, but a nonzero  value in column 1

# Problem 7

Repeat the same calculation with `WarAndPeace.txt` with 2 hidden states. Interpret/explain your results. Which Cyrillic characters appear to be vowels?

In [68]:
symbols, obs = prep_data('WarAndPeace.txt')
h = hmm.CategoricalHMM(n_components=2, n_iter=200, tol=1e-4)
h.fit(obs.reshape(-1,1))
B = h.emissionprob_.T
for i in range(len(B)):
    print(u"{}, {:0.4f}, {:0.4f}"
                .format(symbols[i], *B[i,:]))

 , 0.0877, 0.2146
а, 0.1760, 0.0000
б, 0.0000, 0.0250
в, 0.0000, 0.0655
г, 0.0000, 0.0296
д, 0.0000, 0.0385
е, 0.1427, 0.0180
ж, 0.0000, 0.0140
з, 0.0000, 0.0252
и, 0.1315, 0.0017
й, 0.0000, 0.0149
к, 0.0010, 0.0497
л, 0.0000, 0.0719
м, 0.0000, 0.0381
н, 0.0000, 0.0973
о, 0.2407, 0.0000
п, 0.0062, 0.0346
р, 0.0000, 0.0597
с, 0.0280, 0.0513
т, 0.0000, 0.0780
у, 0.0590, 0.0000
ф, 0.0003, 0.0018
х, 0.0000, 0.0111
ц, 0.0000, 0.0049
ч, 0.0038, 0.0167
ш, 0.0000, 0.0109
щ, 0.0000, 0.0047
ъ, 0.0003, 0.0003
ы, 0.0376, 0.0000
ь, 0.0433, 0.0009
э, 0.0066, 0.0000
ю, 0.0024, 0.0079
я, 0.0328, 0.0128
ё, 0.0001, 0.0000


Once again, we see a split among characters in the 2 columns. It looks like the vowels have a larger numerical value in column 0 than in column 1.

From this, it looks like the vowels are "а, и, o, у, ы, ь, э, я"